# BiG-FAM Query
Summary of [BiG-FAM Query](https://bigfam.bioinformatics.nl/home) results from project: `[{{ project().name }}]`

## Description
Query against 1.2 Million BGCs in the BiG-FAM database using BiG-SLICE

In [ ]:
import pandas as pd
from pathlib import Path
import sqlite3
from IPython.display import display, Markdown, HTML

import json, yaml
import altair as alt
import ast

import warnings
warnings.filterwarnings('ignore')

import networkx as nx
import plotly.graph_objects as go
import altair as alt

In [ ]:
def generate_bigfam_network(df, rank=0):
    """
    Build a networkx graph from bigscape df network
    """
    df[df['rank'] == rank]
    G = nx.from_pandas_edgelist(df, source='bgc_id', target='gcf_id', edge_attr=['membership_value',
           'rank'], edge_key='bigfam_edge_id')
    return G

In [ ]:
def annotate_bigfam_models(G, df, columns = ['bgc_member', 'chemical_class_hits', 'top_chemical_class',
                                             'top_chemical_class_proportion', 'top_chemical_subclass',
                                             'top_chemical_subclass_proportion', 'taxonomic_hits', 
                                             'taxonomic_level', 'H-index', 'richness', 'top_taxa', 
                                             'top_taxa_proportion']):
    for bgc in G.nodes:
        if bgc in df.index:
            G.nodes[bgc]['node_type'] = "BiG-FAM GCFs"
            G.nodes[bgc]['text'] = f"GCF {bgc}<br>size: {df.loc[bgc, 'bgc_member']}<br>top_chemical_class: {df.loc[bgc, 'top_chemical_class']} ({df.loc[bgc, 'top_chemical_class_proportion']:.0%})\
<br>top_chemical_subclass: {df.loc[bgc, 'top_chemical_subclass']} ({df.loc[bgc, 'top_chemical_subclass_proportion']:.0%})\
<br>top_taxa: {df.loc[bgc, 'top_taxa']} ({df.loc[bgc, 'top_taxa_proportion']:.0%})"
            for c in columns:
                G.nodes[bgc][c] = df.loc[bgc, c]
    return G

In [ ]:
def annotate_bigfam_antismash(G, antismash_region_path, columns = ["genome_id", "product", "contig_edge", "region_length", "most_similar_known_cluster_id", "most_similar_known_cluster_description", "most_similar_known_cluster_type", "similarity"]):
    df_antismash = pd.read_csv(antismash_region_path, index_col=0)
    for bgc in G.nodes:
        if bgc in df_antismash.index:
            G.nodes[bgc]['node_type'] = "BGC"
            G.nodes[bgc]['text'] = f"{bgc}<br>{df_antismash.loc[bgc, 'product']}"
            if 'most_similar_known_cluster_description' in df_antismash.columns:
                G.nodes[bgc]['text'] = G.nodes[bgc]['text'] + f"<br>{df_antismash.loc[bgc, 'most_similar_known_cluster_description']}"
            for c in df_antismash.columns:
                if c in columns:
                    G.nodes[bgc][c] = df_antismash.loc[bgc, c]
    return G

In [ ]:
def create_edge_trace(G, name):
    edge_trace = go.Scatter(
        x=[],
        y=[],
        name=name,
        line=dict(width=0.5,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])
    return edge_trace

def create_node_trace(G, node_trace_category, color, showtextlabel=False, nodesize=10, nodeopacity=0.8, nodesymbol="circle", linewidth=1, linecolor="black", textposition="top center"):
    if showtextlabel:
        markermode = "markers+text"
    else:
        markermode = "markers"
    node_trace = go.Scatter(
            x=[],
            y=[],
            text=[],
            textposition=textposition,
            mode=markermode,
            hoverinfo='text',
            name=node_trace_category,
            marker=dict(
                symbol=nodesymbol,
                opacity=nodeopacity,
                showscale=False,
                color=color,
                size=nodesize,
                line=dict(width=linewidth, color=linecolor)))

    for node in G.nodes():
        if G.nodes[node]["node_trace"] == node_trace_category:
            x, y = G.nodes[node]['pos']
            node_trace['x'] += tuple([x])
            node_trace['y'] += tuple([y])
            node_trace['text'] +=tuple([G.nodes[node]['text']])
    return node_trace

In [ ]:
report_dir = Path("../")

with open(report_dir / "metadata/project_metadata.json", "r") as f:
    project_configuration = json.load(f)
with open(report_dir / "metadata/dependency_versions.json", "r") as f:
    dependency_version = json.load(f)

In [ ]:
project_name = [i for i in project_configuration.keys()][0]
antismash_version = dependency_version["antismash"]

query_dir = report_dir / f"bigslice/query_as_{antismash_version}/"
df_bigfam_model = pd.read_csv(query_dir / "gcf_summary.csv")
df_bigfam_hits = pd.read_csv(report_dir / f"bigslice/query_as_{antismash_version}/query_network.csv")
antismash_region_path = report_dir / f"tables/df_regions_antismash_{antismash_version}.csv"
df_annotated_bigfam_model = pd.read_csv(query_dir / "gcf_annotation.csv", index_col=0)
df_network = pd.read_csv(query_dir / "query_network.csv")

In [ ]:
# filtering
#df_network = df_network[df_network["rank"] < 3] # get first degree only
df_annotated_bigfam_model = df_annotated_bigfam_model[df_annotated_bigfam_model["bgc_member"] < 15000] # remove model with really large size
df_network = df_network[df_network["gcf_id"].isin(df_annotated_bigfam_model.index)]

In [ ]:
df_cut = df_annotated_bigfam_model[df_annotated_bigfam_model.top_taxa_proportion <= 0.3]

G = generate_bigfam_network(df_network)
G = annotate_bigfam_antismash(G, antismash_region_path)
G = annotate_bigfam_models(G, df_annotated_bigfam_model)
G_raw = G.copy()

# position nodes
pos = nx.nx_agraph.graphviz_layout(G)
for n, p in pos.items():
    G.nodes[n]['pos'] = p

## Summary Result
### Distribution of the assigned genus from BiG-FAM Model hits

In [ ]:
# Calculate the number of GCFs with representation across different genera
df_genus_dist = pd.DataFrame(index=df_annotated_bigfam_model.index)
for gcf_id in df_annotated_bigfam_model.index:
    taxa = str(df_annotated_bigfam_model.loc[gcf_id, "taxa_distribution"])
    taxa_dict = ast.literal_eval(taxa)
    for genus in taxa_dict.keys():
        df_genus_dist.loc[gcf_id, genus] = taxa_dict[genus]
df_genus_dist.fillna(0, inplace=True)

# Genera represented by most number of GCFs
df_genus_dist_binary = df_genus_dist > 0 
df_genus_dist_binary = df_genus_dist_binary * 1

# Assuming df_genus_dist_binary.sum().sort_values(ascending=False)[:10] is stored in a variable named 'data'
data = df_genus_dist_binary.sum().sort_values(ascending=False)[:10]

# Convert Series to DataFrame for Altair plotting
data_df = data.reset_index()
data_df.columns = ['Genus', 'Counts']

# Create Altair Chart
chart = alt.Chart(data_df).mark_bar().encode(
    y=alt.Y('Genus:N', sort='-x', title='Genera'),
    x=alt.X('Counts:Q', title='Counts'),
    color=alt.Color('Genus:N', legend=None)  # Color by Genera, but we don't need a legend here
).properties(
    title='Top 10 Genus Distribution',
    width=700,
    height=300
)

# Display the chart
chart

### Diversity and Member Size of BiG-FAM model hits

In [ ]:
bgc_hits = len([n for n in G_raw.nodes if G_raw.nodes[n]['node_type'] == 'BGC'])
GCF_hits = len([n for n in G_raw.nodes if G_raw.nodes[n]['node_type'] != 'BGC'])
shannon_non_zero = df_annotated_bigfam_model[df_annotated_bigfam_model['H-index'] != 0]

In [ ]:
shannon_min_non_zero_tax = ast.literal_eval(str(df_annotated_bigfam_model.loc[shannon_non_zero['H-index'].astype(float).idxmin(), "taxa_distribution"]))
shannon_min_non_zero_tax = {k : v/sum(shannon_min_non_zero_tax.values()) for k,v in shannon_min_non_zero_tax.items()}
shannon_min_non_zero_tax_clean = [f"{k} ({v:.1%})" for k,v in sorted(shannon_min_non_zero_tax.items(), key=lambda x:x[1], reverse=True)]

def get_top_taxa(df, gcf):
    result = f"{df.loc[gcf, 'top_taxa']} ({df.loc[gcf, 'top_taxa_proportion'] * df.loc[gcf, 'bgc_member']:.0f} genomes)"
    return result

In [ ]:
mapping = df_bigfam_hits.gcf_id.value_counts()
for gcf in df_annotated_bigfam_model.index:
    df_annotated_bigfam_model.loc[gcf, "dataset_hits"] = mapping[gcf]

In [ ]:
domain = list(df_annotated_bigfam_model[df_annotated_bigfam_model["top_taxa_proportion"] > 0.1]["top_taxa"].value_counts().to_dict().keys())
domain.append("Other")

r = []
range_ = ["#264653", "#287271", "#2a9d8f", "#8ab17d", "#e9c46a", "#f4a261", "#ee8959", "#e76f51", "#edede9"]
for num, d in enumerate(domain):
    if num < len(range_):
        r.append(range_[num])
    else:
        r.append("white")

In [ ]:
source = df_annotated_bigfam_model.copy()
source = source.reset_index().rename(columns={"gcf_id":"BiG-FAM_id"})
for i in source.index:
    if source.loc[i, "top_taxa_proportion"] <= 0.5:
        source.loc[i, "top_taxa"] = "Other"
        
chart_one = alt.Chart(source).mark_point().encode(
    alt.Y('H-index:Q',
          scale=alt.Scale(domain=(-0.5, 5)),
          axis=alt.Axis(title="Shannon Index (H')")
         ),
    alt.X('bgc_member:Q',
          scale=alt.Scale(type="log"),
          axis=alt.Axis(title="Member Size")
         ),
    alt.Size('dataset_hits',
             scale=alt.Scale(type='pow'),# domain=(1, 30)), 
             title="Number of hits in dataset"
            ),
    alt.Color("top_taxa:N", scale=alt.Scale(domain=domain, range=r), title="Top Genus (>=50%)"),
    tooltip=['BiG-FAM_id', 'bgc_member', 'chemical_class_hits', 'top_chemical_class', alt.Tooltip('top_chemical_class_proportion', format='.1%'), 
             'top_chemical_subclass', alt.Tooltip('top_chemical_subclass_proportion', format='.1%'),
             'taxonomic_level', 'richness', 'top_taxa', alt.Tooltip('top_taxa_proportion', format='.1%'), 
             alt.Tooltip('H-index:Q', format='.2f')],
).mark_point(
    filled=True,
    stroke='black',
    strokeWidth=0.5,
    opacity=0.8,
    size=1000
).configure_header(
    title=None,
    labels=False
).configure_axis(
    labelFontSize=10,
    titleFontSize=12
).configure_legend(
    labelFontSize=10,
    titleFontSize=12,
).configure_view(
    continuousHeight=500,
    continuousWidth=500,
).configure_legend(
  orient='right'
)

chart_one

### Network of BiG-FAM model hits

In [ ]:
color_category = 'node_type'
colormap = {"BiG-FAM GCFs" : "red",
           "BGC" : "blue"}
node_trace_category = {}

for node in G.nodes:
    nodeshape = "circle"
    if node in df_annotated_bigfam_model.index:
        G.nodes[node]['node_type'] = "BiG-FAM GCFs"
    if G.nodes[node]['node_type'] == "BiG-FAM GCFs":
        nodeshape = "diamond"
        color = colormap["BiG-FAM GCFs"]
        cat = "BiG-FAM GCFs"
    else:            
        cat = G.nodes[node][color_category]
        color = colormap[cat]
        if 'similarity' in G.nodes[node].keys():
            if G.nodes[node]['similarity'] > 0.8:
                cat = cat + " " + "Known (antiSMASH) > 80% similarity"
                nodeshape = "circle"
            elif G.nodes[node]['similarity'] > 0.4:
                cat = cat + " " + "Known (antiSMASH) > 40% similarity"
                nodeshape = "triangle-up"
            elif G.nodes[node]['similarity'] < 0.4:
                cat = cat + " " + "Unknown (antiSMASH) < 40% similarity"
                nodeshape = "triangle-down"
        else:
            cat = cat + " " + "Unknown"
            nodeshape = "circle"
        
        linecolor = "black"
    
    G.nodes[node]['node_trace'] = cat
    node_trace_category[cat] = {"nodecolor" : color,
                                "nodeshape" : nodeshape,
                                "linecolor" : linecolor}

In [ ]:
edge_trace = create_edge_trace(G, "bigslice_similarity")
traces = [edge_trace]
for cat in node_trace_category.keys():
    nodeopacity = 0.8
    color = node_trace_category[cat]["nodecolor"]
    nodeshape = node_trace_category[cat]["nodeshape"]
    linecolor = node_trace_category[cat]["linecolor"]
    node_trace = create_node_trace(G, cat, color, nodesymbol=nodeshape, nodeopacity=nodeopacity, nodesize=10, linewidth=1, linecolor=linecolor)
    traces.append(node_trace)
    
fig2 = go.Figure(data=traces,
                layout=go.Layout(
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    showlegend=True,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    width=1200, height=800))

outfile = Path("assets/figures/bigfam-query.html")
outfile.parent.mkdir(parents=True, exist_ok=True)
fig2.write_html(outfile)

display(HTML(filename=str(outfile)))

## References
<font size="2">
{% for i in project().rule_used['query-bigslice']['references'] %}
  - *{{ i }}*
{% endfor %}
</font>